# MATH 104 - PSET 4 - Question 5 Part C
## Jack Le - Winter 2023

In [4]:
import numpy as np
import time

## Helper functions to multiply diagonal matrices
### Implements it as matrix-vector multiplicaiton

In [5]:
# Used when diagonal matrix is on the left, scales rows
def diag_row_mult(diag, A):
    res = np.empty(A.shape, float)
    for i, row in enumerate(A):
        # Scale each row by the corresponding diagonal entry
        res[i, :] = row * diag[i]
    return res

In [6]:
# Used when the diagonal matrix is on the right, scales columns
def diag_col_mult(diag, A):
    res = np.empty(A.shape, float)
    for i in range(A.shape[1]):
        # Scale each column by the corresponding diagonal entry
        res[:, i] = A[:, i] * diag[i]
    return res

## Implementation of Sherman-Morrison-Woodburry Formula

$$(D + FF^\top)^{-1} = D^{-1}-D^{-1}F(I_{10} + F^\top D^{-1}F)^{-1}F^\top D^{-1}$$

This is specific to a 10000x10000 diagonal matrix D and a 10000x10 matrix F.
There will comments on the number of flops for each operation.
Flops are floating-point operations, defined as multiplication, addition, subtraction, etc

Directly computing inverses will cost about $\frac{8}{3}n^3$.

In [4]:
# Computes inverse of D + F @ F.T
# D is 10,000 x 10,000. F is 10,000 x 10.
# Flops are floating-point operations, defined as multiplication, addition, subtraction, etc
def smw_inverse(D, F):
    # O(n) to get the n diagonal entries
    # 0 flops to read entries
    diag = np.diagonal(D)
    
    # O(n) to reciprocal each entry
    # 10,000 flops to reciprocal each diagonal entry
    diag_inv = 1/diag 
    
    # O(n^2) to go through every row and scale it by diagonal entry
    # There's 100,000 entries in F, each being scaled.
    # 100,000 flops to scale every entry in F
    # This matrix is now 10,000x10
    diag_inv_times_f = diag_row_mult(diag_inv, F) 
    
    # O(n^3) for matrix multiplication
    
    # For matrix mult, each entry is the dot product of two
    # 10,000-vectors. This means 10,000 flops to multiply
    # and 10,000 flops to add those together.
    # There are 10 * 10 = 100 entries, so the matrix mult
    # will require 2,000,000 flops
    
    # We then sum the two 10x10 matrices, so 100 additional flops
    inside = np.identity(10) + F.T @ diag_inv_times_f
    # This will cost roughly 8/3 (10)^3 or 2667 flops
    inside_inv = np.linalg.inv(inside)
    
    # O(n^2) to go through every col and scale it by diagonal entry
    # There's 100,000 entries in F.T, each being scaled.
    # 100,000 flops to scale every entry in columns
    # This matrix is now 10x10,000
    f_t_times_d_inv = diag_col_mult(diag_inv, F.T)
    
    # We now want to compute diag_inv_times_f @ inside_inv @ f_t_times_d_inv
    # I will break this up to better comment on flops
    
    # O(n^3) for matrix multiplications
    
    # This is multiplication between a  10,000x10 and a 10x10 matrix
    # Each entry in first_mult is the dot product of two 10-vectors. This thus
    # requires 20 flops to compute. There are 10,000 * 10 = 100,000 entries 
    # in first_mult. Thus, this requires 2,000,000 flops to make first_mult.
    first_mult = diag_inv_times_f @ inside_inv
    
    # This is multiplication between a  10,000x10 and a 10x10000 matrix
    # Each entry in right_side is the dot product of two 10-vectors. This thus
    # requires 20 flops to compute. There are 10,000 * 10,000 = 100,000,000 entries
    # in right_side. Thus, this requires 2,000,000,000 flops to make right_side.
    right_side = first_mult @ f_t_times_d_inv
    
    # O(n) to create zero matrix with diagonal entries
    # 0 flops to create matrix and copy entries
    D_inv = np.diag(diag_inv) 
    
    # We then subtract the two 10000x10000 matrices, so 100,000,000 additional flops
    return D_inv - right_side

In total, we see that finding the inverse using the Sherman-Morrison-Woodburry formula requires
about 2,104,212,767 flops.

D + FF^T is a 10000x10000 matrix. Since directly computing inverses will cost about $\frac{8}{3}n^3$ flops, if we were to directly compute the inverse of D + FF^T, it would cost about $\frac{8}{3}10000^3$ = 2,666,666,666,667 flops. This is orders of magnitude more than what was required using the Sherman-Morrison-Woodburry formula.

# Testing

In [5]:
# Create D, our diagonal matrix
np.random.seed(104)
diag = np.random.randint(1, 1000, 10000)
D = np.diag(diag)
print(D.shape)
print(D)

(10000, 10000)
[[ 70   0   0 ...   0   0   0]
 [  0 706   0 ...   0   0   0]
 [  0   0 730 ...   0   0   0]
 ...
 [  0   0   0 ...  73   0   0]
 [  0   0   0 ...   0 934   0]
 [  0   0   0 ...   0   0 437]]


In [6]:
# Create the F matrix
np.random.seed(94305)
F = np.random.randint(0, 1000, size = (10000, 10))
print(F.shape)
print(F)

(10000, 10)
[[ 15  30 769 ...  17   4 372]
 [441 867 330 ... 768 950 688]
 [250 966 849 ... 625 182 279]
 ...
 [326 910 709 ... 856 151 968]
 [280 913 807 ... 221 609 190]
 [291 599 181 ...  95 118 727]]


In [7]:
start_time = time.time()
inv_smw = smw_inverse(D, F)
print("--- %.3f seconds ---" % (time.time() - start_time))

--- 0.654 seconds ---


In [8]:
test_matrix = D + F @ F.T
start_time = time.time()
inv_T = np.linalg.inv(test_matrix)
print("--- %.3f seconds ---" % (time.time() - start_time))

--- 10.624 seconds ---


In [9]:
# Check if our inverses are correct
print("--- All correct : %s ---" % np.allclose(inv_T, inv_smw))

--- All correct : True ---


# Stress Testing

In [10]:
def test_function(cycles):
    max_time_smw = max_time_inv = -10e9
    min_time_smw = min_time_inv = 10e9
    avg_time_smw = avg_time_inv = 0
    
    correct = True
    
    for i in range(cycles):
        # Create matrices
        diag = np.random.randint(1, 1000, 10000)
        D = np.diag(diag)
        F = np.random.randint(0, 1000, size = (10000, 10))
        
        # Test SMW
        start_time = time.time()
        inv_smw = smw_inverse(D, F)
        runtime = time.time() - start_time
        max_time_smw = max(max_time_smw, runtime)
        min_time_smw = min(min_time_smw, runtime)
        avg_time_smw += runtime
        
        # Test actual inverse
        test_matrix = D + F @ F.T
        start_time = time.time()
        inv_T = np.linalg.inv(test_matrix)
        runtime = time.time() - start_time
        max_time_inv = max(max_time_inv, runtime)
        min_time_inv = min(min_time_inv, runtime)
        avg_time_inv += runtime
        
        # Use allclose because of floating point precision
        correct &= np.allclose(inv_T, inv_smw)
        
        if not correct:
            print(inv_T)
            print(inv_smw)
            return

    avg_time_inv /= cycles
    avg_time_smw /= cycles
    
    print("--- Time for direct inverse ---")
    print("--- Average: %.3f seconds ---" % (avg_time_inv))
    print("--- Max: %.3f seconds ---" % (max_time_inv))
    print("--- Min: %.3f seconds ---" % (min_time_inv))
    
    print()
        
    print("--- Time for Sherman-Morrison-Woodbury inverse ---")
    print("--- Average: %.3f seconds ---" % (avg_time_smw))
    print("--- Max: %.3f seconds ---" % (max_time_smw))
    print("--- Min: %.3f seconds ---" % (min_time_smw))
    
    print()
    
    print("--- Time difference between direct and SMW inverse ---")
    print("--- Average: %.3f seconds ---" % (avg_time_inv - avg_time_smw))
    print("--- Max: %.3f seconds ---" % (max_time_inv - max_time_smw))
    print("--- Min: %.3f seconds ---" % (min_time_inv - min_time_smw))
    
    print()
    
    print("--- How much longer does direct inverse take ---")
    print("--- Average fac: %.3f times longer ---" % (avg_time_inv / avg_time_smw))
    print("--- Max fac: %.3f times longer ---" % (min_time_inv / min_time_smw))
    print("--- Min fac: %.3f times longer ---" % (max_time_inv / max_time_smw))

    print()
    
    print("--- Every inverse is correct: %s ---" % correct)
    

In [11]:
# We run 10 cycles of the test function to account for possible outliers
test_function(10)

--- Time for direct inverse ---
--- Average: 10.677 seconds ---
--- Max: 11.502 seconds ---
--- Min: 10.044 seconds ---

--- Time for Sherman-Morrison-Woodbury inverse ---
--- Average: 0.371 seconds ---
--- Max: 0.502 seconds ---
--- Min: 0.337 seconds ---

--- Time difference between direct and SMW inverse ---
--- Average: 10.306 seconds ---
--- Max: 10.999 seconds ---
--- Min: 9.707 seconds ---

--- How much longer does direct inverse take ---
--- Average fac: 28.766 times longer ---
--- Max fac: 29.793 times longer ---
--- Min fac: 22.896 times longer ---

--- Every inverse is correct: True ---
